# Test your CNN
Here, we are going to test your CNN on data that it has never seen before. You will all test your CNN on the same dataset, so it will be a fair competition.

The winner will be the CNN that produces the highest classification accuracy on the test set. In the unlikely situation that two or more CNNs have the same accuracy, then the winner will be the CNN that scores the lowest loss.

In [7]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, random_split

# Define the convolutional neural network
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2) # 28 input, 28 output
        self.conv2 = nn.Conv2d(16, 32, kernel_size=4, padding=2, stride=2) # 28 input, 14 output
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 14 input, 14 output (+pool -> 7 output)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 7 input, 7 output (+pool -> 3 output)
        self.conv5 = nn.Conv2d(128, 10, kernel_size=3, padding=0) # 3 input, 1 output (+pool -> 1 output)

    def forward(self, x):
        x = torch.relu(self.conv1(x)) # 28 input, 28 output
        # x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x)) # 28 input, 14 output
        # x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x)) # 14 input, 14 output (+pool -> 7 output)
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv4(x)) # 7 input, 7 output (+pool -> 3 output)
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv5(x)) # 3 input, 1 output (+pool -> 1 output for each class) 
        x = x.squeeze(-1)  # Now shape is (batch_size, number of classes)
        return x

# Setup the loss
criterion = nn.CrossEntropyLoss() # Loss function

#Download and process the test dataset
class PadTranslateCrop:
    def __init__(self, pad, translate_px):
        self.pad = pad
        self.translate_px = translate_px

    def __call__(self, img):
        # Pad the image
        img = transforms.functional.pad(img, self.pad, fill=0)
        # Translate the image
        dx = torch.randint(-self.translate_px, self.translate_px + 1, (1,)).item()
        dy = torch.randint(-self.translate_px, self.translate_px + 1, (1,)).item()
        img = transforms.functional.affine(img, angle=0, translate=(dx, dy), scale=1, shear=0, fill=0)
        # Crop back to original size
        w, h = img.size
        left = (w - 28) // 2
        top = (h - 28) // 2
        img = transforms.functional.crop(img, top, left, 28, 28)
        return img

# Usage in your transform pipeline
custom_transform = transforms.Compose([
    PadTranslateCrop(pad=14, translate_px=10),  # pad and allow up to 10px translation
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and prepare the training and test datasets
data_test = datasets.MNIST(root='./data', train=False, download=True, transform=custom_transform)
test_loader = DataLoader(data_test, batch_size=data_test.data.shape[0])
# Get a batch of images and labels from the train_loader
images, labels = next(iter(test_loader))
# Load your CNN
model = CNN()
model.load_state_dict(torch.load('./model_trained'))
model.eval()

# Get predictions
with torch.no_grad():
    outputs = model(images).squeeze(-1)
    loss = criterion(outputs, labels)  # Calculate loss for the batch

# Calculate classification accuracy
_, predicted = torch.max(outputs, 1)  # Detach outputs before using for accuracy
correct = (predicted == labels).sum().item() # Sum number of correct predictions
accuracy = correct / labels.size(0) * 100.0 # Calculate percentage accuracy

# Print the accuracy and loss
print(f'Accuracy: {accuracy:.2f}%   |   Loss: {loss.item():.4f}')

Accuracy: 92.79%   |   Loss: 0.2241
